<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/BERTopic_seven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 1.6 MB/s 
     |████████████████████████████████| 5.2 MB 11.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 6.9 MB/s 
     |████████████████████████████████| 85 kB 5.3 MB/s 
     |████████████████████████████████| 4.4 MB 59.3 MB/s 
     |████████████████████████████████| 1.2 MB 57.2 MB/s 
     |████████████████████████████████| 101 kB 11.5 MB/s 
     |████████████████████████████████| 636 kB 64.2 MB/s 
     |████████████████████████████████| 6.6 MB 40.6 MB/s 
     |████████████████████████████████| 1.1 MB 28.6 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2342108 sha256=49bebb41439602318dd3fa40622ad1855e129d252d47f938e17397aaef1ca6b0
  Stored in directory: /root/.cache

In [19]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/BERTopic/corona.csv ./corona.csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import pandas as pd
docs_list = []
csv_pd = pd.read_csv(filepath_or_buffer="./corona.csv", encoding="cp932", sep=",")
lists_pd = csv_pd[csv_pd.columns[1]]
lists = list(lists_pd.values)
for lst in lists:
    docs_list = docs_list + lst.rstrip('。').split("。")
    

docs_pd = pd.DataFrame(docs_list, columns=[csv_pd.columns[1]])
docs = docs_pd[csv_pd.columns[1]].tolist()

In [39]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.cluster import KMeans
cluster_model = KMeans(n_clusters=20)
#from sklearn.mixture import GaussianMixture
#cluster_model = GaussianMixture(n_components=3)

sentence_model = SentenceTransformer("sonoisa/t5-base-japanese")
#topic_model = BERTopic(embedding_model=sentence_model, calculate_probabilities=True, verbose=True, nr_topics="20")
topic_model = BERTopic(embedding_model=sentence_model, calculate_probabilities=True, verbose=True)
#topic_model = BERTopic(embedding_model=sentence_model, hdbscan_model=cluster_model, calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/sonoisa_t5-base-japanese. Creating a new one with MEAN pooling.


Batches:   0%|          | 0/354 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
2022-06-29 12:34:20,545 - BERTopic - Transformed documents to Embeddings
2022-06-29 12:34:39,878 - BERTopic - Reduced dimensionality
2022-06-29 12:35:04,499 - BERTopic - Clustered reduced embeddings


In [40]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,4053,-1_コロナ_そして_でも_共に頑張ろう
1,0,442,0_また_私は_正直_感染に対する政治的処置
2,1,338,1_スーパー_本当にありがとうございます_医療従事者の方_ありがとうございます
3,2,265,2_どうか_必ず_今は我慢のとき_11等とはまた異なり
4,3,186,3_外出するな_家から出るな_出歩くな_外出を控えましょう
...,...,...,...
188,187,10,187_忍耐_コロナ予防_自分で考える_我慢忍耐根性
189,188,10,188_じっと耐える_耐え忍ぼう_耐えがたきを耐える_無理せず耐える
190,189,10,189_北海道を見習って_乗り換えよう_ワンチームの精神で_トップをかえて
191,190,10,190_乗り越えましょう_とにかく乗り越えるしかない_乗り越えよう_乗り越えたい


In [41]:
topic_model.get_topic(0)

[('また', 0.008807513358515671),
 ('私は', 0.006460276339981064),
 ('正直', 0.00593182028415344),
 ('感染に対する政治的処置', 0.00425845821821559),
 ('支出は増えて', 0.00425845821821559),
 ('政治はしないこと', 0.00425845821821559),
 ('政府は国民が今経済に苦しんでいることをぜんぜんわかろうとしていない', 0.00425845821821559),
 ('戦争中はもっと大変な我慢を強いられていたはず', 0.00425845821821559),
 ('政府や著名人をパッシングしたり', 0.00425845821821559),
 ('我慢したり', 0.00425845821821559)]

In [42]:
topics_pd = pd.DataFrame(topics, index=docs_pd.index, columns=['topics'])
topics_pd = pd.concat([topics_pd, docs_pd], axis=1)
topics_pd

,topics,いま、世の中や企業に一番届けたい気持ちを教えてください！【Q2S1 【 】】
0,44,普通の今まで通りの生活をしたいし、いく予定だった旅行やライブに行きたい！
1,40,頑張れ
2,0,お出かけしたい気持ちはよく分かりますが、その行動が誰かの命を奪うことになるのでちゃんと自粛し...
3,-1,企業の皆さん、大変なこと、辛いこといっぱいあると思います
4,1,恐怖の中一生懸命社会を支えてくれてありがとうございます
...,...,...
11313,6,このご時世、(思いやり)の精神を失ったら、因果応報です
11314,-1,今は皆で助け合い知恵を出してコロナに打ち勝とう
11315,8,コロナウイルスに負けるな
11316,38,落ち着いて、現実に向き合い、良き未来になるよう行動すること


In [43]:
topics_pd[topics_pd['topics'] == 0]

,topics,いま、世の中や企業に一番届けたい気持ちを教えてください！【Q2S1 【 】】
2,0,お出かけしたい気持ちはよく分かりますが、その行動が誰かの命を奪うことになるのでちゃんと自粛し...
59,0,スーパーで働いているが、客が多すぎて感染しそうで毎日冷や冷やしていて、早く対策を始めてほしい
112,0,でも、当初予定されていた入学式やオリエンテーションが中止となり、更には授業開始も延期されてし...
233,0,不要不急の外出は控えています！でもいざ外出する時につけるマスクがありません…少しでもいいので...
251,0,患者が多いからで即仕事しなきゃという環境はおかしい
...,...,...
11008,0,今の状況は、特定の人だけが直面しているのではなく、我々全員が共通して直面している苦難である
11047,0,今こそ島国根性を捨てようや、ほんまに日本沈没するで、沈没せんでも弱った所をチャイナにやられてまうで
11118,0,感染に対する政治的処置、対策が間違っている
11156,0,誰もが、追い詰められたような状況下にありますが、こんな時だからこそ人間であることを自覚したい...


In [44]:
topic_model.visualize_topics()

In [45]:
topic_model.visualize_distribution(probs[200], min_probability=0.0)

In [46]:
topic_model.visualize_hierarchy()

In [48]:
topic_model.visualize_barchart(top_n_topics=20)

In [49]:
topics_pd.to_csv('./BERTopic_sonoisa.csv', sep=',', encoding='cp932')
!cp ./BERTopic_sonoisa.csv /content/drive/MyDrive/BERTopic/